### Install Libraries

In [1]:
!pip install -Uqq pypdfium2
!pip install -Uqq tabula-py
!pip install -Uqq pdfplumber
!pip install -Uqq python-dateutil
!pip install -Uqq xlrd

### Load Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pypdfium2 as pdfium
import pandas as pd
import numpy as np
import os.path
import tabula
from tabula.io import read_pdf
import pdfplumber
import re

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

import uuid

### Define Functions

#### Tera Nova Extract Text Fields

In [3]:
#### Tera Nova Extract Text Fields# This function takes pdf as an input
# converts into text
# returns the required fields
    
def TeraNovaText(folder,file,prjct_id,scrpr_id):
    pdf_fium = pdfium.PdfDocument(folder+'/'+file) # Read the PDF as a text File
    pdf_plumb = pdfplumber.open(folder+'/'+file) # Read the PDF as a text File
    
    no_of_pages = len(pdf_fium)
    
    out= []
    
    for i in range(no_of_pages):
        temp_out = []
        
        page_fium = pdf_fium[i] # pdfium
        page_plumb = pdf_plumb.pages[i] # PDF Plumber
        
        # Load a text page helper
        textpage_fium = page_fium.get_textpage() # pdfium
    
        # Extract text from the whole page
        text_fium = textpage_fium.get_text_range() # pdfium
        text_plumb = page_plumb.extract_text() # PDF Plumber
    
        # Split the list with the delimiters
        list_fium = text_fium.split('\r\n') # pdfium
        list_plumb = text_plumb.split('\n')  # PDF Plumber
    
    
        # Store the requird fields
    
        # Data Ingestion_datetime
        Ingestion_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
    
        # City
        #City = list_plumb[3].split(',')[0].split(' ')[-1]
    
        # DATE MOLDED
        date_molded_src = "DATE MOLDED"
        date_molded_src_get_string = [x for x in list_fium if date_molded_src in x]
        date_molded_src_lst_str = ''.join(date_molded_src_get_string)
        Date_Molded = date_molded_src_lst_str.split(' ')[2]
    
        # DATE ISSUED
        date_issue_src = "DATE ISSUED"
        date_issue_src_get_string = [x for x in list_fium if date_issue_src in x]
        date_issue_src_lst_str = ''.join(date_issue_src_get_string)
        Date_Issued = date_issue_src_lst_str.split(':')[1].strip()
    
        # LAB NUMBER
        Lab_Number = date_molded_src_lst_str.split(' ')[5]
    
        # LOCATION OF PLACEMENT
        colon = ":"
        lop_searc = "LOCATION OF PLACEMENT"
        lop_searc_get_string = [x for x in list_fium if lop_searc in x]
        lop_searc_lst_str = ''.join(lop_searc_get_string)

        loc = lop_searc_lst_str

        if any(c in colon for c in loc):
            Location_of_Placement = loc.split(':')[1].strip()
        else:
            loc = loc.split(' ')
            loc.insert(3, ':')
            loc = " ".join(loc) 
            Location_of_Placement = loc.split(':')[1].strip()
        
        # CONCRETE SUPPLIER
        supplier_src = "CONCRETE SUPPLIER"
        supplier_src_get_string = [x for x in list_plumb if supplier_src in x]
        supplier_src_lst_str = ''.join(supplier_src_get_string)
        Concrete_Supplier = supplier_src_lst_str.split('WATER')[0].split(':')[1].strip()
        
        # MIX ID NO
        mixid_src = "MIX ID NO"
        mixid_src_get_string = [x for x in list_plumb if mixid_src in x]
        mixid_src_lst_str = ''.join(mixid_src_get_string)
        Mix_ID_No = mixid_src_lst_str.split('UNIT')[0].split(':')[1].strip()
    
        # WEATHER
        weather_src = "WEATHER"
        weather_src_get_string = [x for x in list_fium if weather_src in x]
        if weather_src_get_string:
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
            
        else:
            weather_src = "W EATHER"
            weather_src_get_string = [x for x in list_fium if weather_src in x]
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
    
        # TIME MOLDED
        time_src = "TIME MOLDED"
        time_src_get_string = [x for x in list_plumb if time_src in x]
        time_src_lst_str = ''.join(time_src_get_string)
        Time_Molded = time_src_lst_str.split(' ')[2] + ' ' +time_src_lst_str.split(' ')[3]
        if "ASTM" in Time_Molded:
            Time_Molded = ""
        else:
            Time_Molded = Time_Molded[0:8]
    
        # AIR CONTENT(%)
        aircontent_src = "AIR CONTENT"
        aircontent_src_get_string = [x for x in list_fium if aircontent_src in x]
        aircontent_src_lst_str = ''.join(aircontent_src_get_string)
        Air_Content = aircontent_src_lst_str.split(':')[1].strip()
        Air_Content = re.sub('[^0-9,.]', '', Air_Content)
    
        # SLUMP(IN)
        slump_src = "SLUMP(IN)"
        slump_src_get_string = [x for x in list_fium if slump_src in x]
        slump_src_lst_str = ''.join(slump_src_get_string)
        Slump_space_chck = slump_src_lst_str.split(':')
        if len(Slump_space_chck[1])>5:
            Slump = Slump_space_chck[1].split(' ')[1].strip()
        else:
            Slump = Slump_space_chck[1].strip()
        
        Slump = re.sub('[^0-9,.]', '', Slump)
        
    
        # SIZE & REQUIRED PSI 
        size_psi_search = "SIZE:"
        size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
        if size_psi_get_string:
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split(':')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()
        else:
            size_psi_search = "SIZE"
            size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split('SIZE')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()
        
        # WATER ADDED(GALS)    
        water_searc = "GALS"
        water_searc_get_string = [x for x in list_fium if water_searc in x]
        water_searc_lst_str = ''.join(water_searc_get_string)
        Water_Added_len_check = water_searc_lst_str.split(':')
        
        if len(Water_Added_len_check[1])>5:
            Water_Added = Water_Added_len_check[1].split(' ')[1].strip()
        else:
            Water_Added = Water_Added_len_check[1].strip()
        
        Water_Added = re.sub('[^0-9,.]', '', Water_Added)
    
    
        # UNIT WEIGHT(PCF):
        unit_searc = "PCF"
        unit_searc_get_string = [x for x in list_fium if unit_searc in x]
        unit_searc_lst_str = ''.join(unit_searc_get_string)
        Unit_Weight = unit_searc_lst_str.split(':')[1].strip()
        Unit_Weight = re.sub('[^0-9,.]', '', Unit_Weight)
    
        # AMBIENT TEMP(F):
        ambi_temp_searc = "AMBIENT TEMP"
        ambi_temp_searc_get_string = [x for x in list_fium if ambi_temp_searc in x]
        ambi_temp_searc_lst_str = ''.join(ambi_temp_searc_get_string)
        Ambient_Temp = ambi_temp_searc_lst_str.split(':')[1].strip()
        Ambient_Temp = re.sub('[^0-9]', '', Ambient_Temp)
    
        # CONCRETE TEMP(F):
        concrete_temp_searc = "CONCRETE TEMP"
        concrete_temp_searc_get_string = [x for x in list_fium if concrete_temp_searc in x]
        concrete_temp_searc_lst_str = ''.join(concrete_temp_searc_get_string)
        Concrete_Temp = concrete_temp_searc_lst_str.split(':')[1].strip()
        Concrete_Temp = re.sub('[^0-9]', '', Concrete_Temp)
        
        # PROJECT ID
        ProjectID = prjct_id
        
        # SCRAPER ID
        ScraperID = scrpr_id
        
        temp_out = [ProjectID,ScraperID,file,Ingestion_datetime,Date_Molded,Date_Issued,Lab_Number,Location_of_Placement
       ,Concrete_Supplier,Mix_ID_No,Weather,Time_Molded,Air_Content,Slump,Size
       ,Required_PSI,Water_Added,Unit_Weight,Ambient_Temp,Concrete_Temp]
        
        out.append((temp_out))
    
    return out


#### Tera Nova Extract Test Report Table

In [4]:
# This function takes pdf as an input
# converts into a dataframe
# returns the test results as a DataFrame

def TeraNovaPdf2Table(folder,file,result):
    readpdf2df = tabula.io.read_pdf(folder+'/'+file 
                             , pages='all'
                             #,output_format="dataframe"
                                   , multiple_tables=True)
    
    df = pd.concat(readpdf2df)
    age_load_read = df.iloc[1:,3].dropna().reset_index().drop(columns=['index']).iloc[:-1,]
    
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('diameter', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('compression', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('age', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    
    # Extra Space in the Data Check
    space_check = age_load_read.iloc[:,0].str.split(' ').tolist()
    
    space_check_ref = []
    for i in space_check:
        for j in i:
            if len(i)>4:
                i[1]=i[1]+i[2]
                i.pop(2)
        space_check_ref.append(i)
    space_check = space_check_ref
    age_load_read = pd.DataFrame(space_check, columns =['Age_Days','Date_Tested','Total_load', 'Unit_load'])
    final_df = pd.DataFrame()
    for i in range(len(result)):
        df_split = np.array_split(age_load_read, len(result))
        
        result_lab = result[i][6]
        result_date_isssue = result[i][5]
        
        df_sel = df_split[i]
        
        iterat = df_sel.shape[0]
        
        lab_nbr = pd.DataFrame({'Lab_Number': result_lab}, index=[0])
        date_isu = pd.DataFrame({'Date_Issued': result_date_isssue}, index=[0])
        
        lab_nbr_itr = pd.DataFrame(np.repeat(lab_nbr.values, iterat, axis=0))
        date_isu_itr = pd.DataFrame(np.repeat(date_isu.values, iterat, axis=0))
        
        lab_nbr_itr.columns = lab_nbr.columns
        date_isu_itr.columns = date_isu.columns
        
        temp_df = pd.concat([lab_nbr_itr,date_isu_itr,age_load_read.iloc[:, age_load_read.columns != 'Total_load']], axis=1, join='inner')
        final_df = final_df.append(temp_df).reset_index(drop=True)
        
    
    return final_df
    
    
    

#### TeraNova Main Function

In [5]:
def TeraNova(folder_dir,prjct_id,scrpr_id):
    log = []
    df_text = []
    pdf_df = pd.DataFrame()

    #path = '/Users/prateek/Documents/Captsone Project/Jupiter Files/TestFolder'
    
    #os.chdir('C:/Users/pkakkar/Documents/2023-02-17-DataSet-006-008')
    #path = 'C:/Users/pkakkar/Documents/2023-02-17-DataSet-006-008'
    
    os.chdir(folder_dir)
    path = folder_dir


    for root, dirs, files in os.walk(path,topdown=True):
        if not root.endswith('.DS_Store'):
            for file in files:
                if file.endswith('.pdf'):
                    read_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
                    folder = os.path.basename(root)
                    dir_path = os.path.dirname(root)
                
                    log.append((prjct_id,scrpr_id,dir_path,folder,file,read_datetime))  # Appends results for log DF
                
                    result = TeraNovaText(root,file,prjct_id,scrpr_id) # Calls text function
                
                    for j in range(len(result)):
                        df_text.append((result[j]))    # Appends results from text function
                     
                    df_out = TeraNovaPdf2Table(root,file,result) # Calls Dataframe function
                    pdf_df = pdf_df.append(df_out).reset_index(drop=True) # Appends test results DF
                

    # Append Log DataFrame        
    df_log = pd.DataFrame(log, columns=['ProjectID', 'ScraperID','Directory','Folder','File', 'Read_DateTime'])

    # Append DataFrame with Text fields
    df_page = pd.DataFrame(df_text, columns=['ProjectID',
                     'ReportScraperID',
                     'File_Name',
                     'Ingestion_datetime',
                     'Date_Molded',
                     'Date_Issued',
                     'Lab_Number',
                     'Location_of_Placement',
                     'Concrete_Supplier',
                     'Mix_ID_No',
                     'Weather',
                     'Time_Molded',
                     'Air_Content(%)',
                     'Slump(in)',
                     'Size',
                     'Required_PSI',
                     'Water_Added(GALS)',
                     'Unit_Weight(PCF)',
                     'Ambient_Temp(F)',
                     'Concrete_Temp(F)'])
    
    return df_page,pdf_df,df_log

#### Tera Nova Data Transformation

In [6]:
def TeraNovaDataTransform(TeraNovaFieldReport, TeraNovaTestResult):
    ############## Create a copy of the Dataframes  #############

    TeraNovaFieldReport_stg = TeraNovaFieldReport.copy()
    TeraNovaTestResult_stg = TeraNovaTestResult.copy()
    
    # generate the uuid for BatchLabID 
    batchlabID = {Lab_Number: str(uuid.uuid4()) for Lab_Number in TeraNovaFieldReport_stg['Lab_Number'].unique()}
    
    # map uuid
    TeraNovaFieldReport_stg['BatchLabID'] = TeraNovaFieldReport_stg['Lab_Number'].map(batchlabID)
    
    # Add BatchLabID to Results Table
    TeraNovaTestResult_stg['BatchLabID'] = TeraNovaTestResult_stg['Lab_Number'].map(batchlabID)
    
    
    ###################### Rename the Columns ########################

    TeraNovaFieldReport_stg = TeraNovaFieldReport_stg.rename(columns={'File_Name': 'ReportFileName'
                                              , 'Date_Issued': 'ReportDateIssued'
                                              , 'Ingestion_datetime': 'ReportTimeIngested'
                                              , 'Ambient_Temp(F)': 'SiteTemperature'
                                              , 'Weather': 'SiteWeather'
                                              , 'Concrete_Supplier': 'ConcreteSupplier'
                                              , 'Location_of_Placement': 'ConcretePlacementLocation'
                                              , 'Lab_Number': 'BatchLabNumber'
                                              , 'Mix_ID_No': 'BatchMixID'
                                              , 'Size': 'BatchSpecimenSize'
                                              , 'Concrete_Temp(F)': 'BatchTemperature'
                                              , 'Unit_Weight(PCF)': 'BatchUnitWeight'
                                              , 'Required_PSI': 'BatchRequiredStrength'
                                              , 'Air_Content(%)': 'BatchAirContent'
                                              , 'Slump(in)': 'BatchtSlump'
                                              , 'Water_Added(GALS)': 'BatchWaterAdded'})



    TeraNovaTestResult_stg = TeraNovaTestResult_stg.rename(columns={'Lab_Number': 'BatchLabNumber'
                                              , 'Date_Issued': 'ReportDateIssued'
                                              , 'Date_Tested': 'SpecimenDateTested'
                                              , 'Unit_load': 'SpecimenMeasuredStrength'
                                              , 'Age_Days': 'SpecimenAgeTested'})



    ########### Changing Data Types of Columns for SpecimenFieldReport #########

    TeraNovaFieldReport_stg['ReportTimeIngested'] = TeraNovaFieldReport_stg['ReportTimeIngested'].astype('datetime64[ns]')
    TeraNovaFieldReport_stg['ReportDateIssued'] = pd.to_datetime(TeraNovaFieldReport_stg["ReportDateIssued"], format="%d-%b-%y")


    TeraNovaFieldReport_stg['SiteTemperature'] = TeraNovaFieldReport_stg['SiteTemperature'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchTemperature'] = TeraNovaFieldReport_stg['BatchTemperature'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchUnitWeight'] = TeraNovaFieldReport_stg['BatchUnitWeight'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchRequiredStrength'] = TeraNovaFieldReport_stg['BatchRequiredStrength'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchAirContent'] = TeraNovaFieldReport_stg['BatchAirContent'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchtSlump'] = TeraNovaFieldReport_stg['BatchtSlump'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchWaterAdded'] = TeraNovaFieldReport_stg['BatchWaterAdded'].replace(r'^\s*$', np.nan, regex=True)

    #df_page_stg['BatchLabNumber'] = df_page_stg['BatchLabNumber'].astype('int')
    TeraNovaFieldReport_stg['SiteTemperature'] = TeraNovaFieldReport_stg['SiteTemperature'].astype('float')
    TeraNovaFieldReport_stg['BatchTemperature'] = TeraNovaFieldReport_stg['BatchTemperature'].astype('float')
    TeraNovaFieldReport_stg['BatchUnitWeight'] = TeraNovaFieldReport_stg['BatchUnitWeight'].astype('float')
    TeraNovaFieldReport_stg['BatchRequiredStrength'] = TeraNovaFieldReport_stg['BatchRequiredStrength'].astype('float')
    TeraNovaFieldReport_stg['BatchAirContent'] = TeraNovaFieldReport_stg['BatchAirContent'].astype('float')
    TeraNovaFieldReport_stg['BatchtSlump'] = TeraNovaFieldReport_stg['BatchtSlump'].astype('float')
    TeraNovaFieldReport_stg['BatchWaterAdded'] = TeraNovaFieldReport_stg['BatchWaterAdded'].astype('float')


    TeraNovaFieldReport_stg["Time_Molded"] = TeraNovaFieldReport_stg['Time_Molded'].str.lower()
    TeraNovaFieldReport_stg['Time_Molded'] = TeraNovaFieldReport_stg['Time_Molded'].replace({'sm':'am', 'om':'pm'}, regex = True)


    TeraNovaFieldReport_stg["BatchTimeMolded"] = TeraNovaFieldReport_stg["Date_Molded"] + ' ' +TeraNovaFieldReport_stg["Time_Molded"]
    TeraNovaFieldReport_stg['BatchTimeMolded'] = TeraNovaFieldReport_stg['BatchTimeMolded'].astype('datetime64[ns]')



    ########### Changing Data Types of Columns for SpecimenCompressionTestResult #########

    # Remove rows with not integer age days
    TeraNovaTestResult_stg = TeraNovaTestResult_stg[pd.to_numeric(TeraNovaTestResult_stg['SpecimenAgeTested'], errors='coerce').notna()]

    # Change ReportedDateIssues to datetime
    TeraNovaTestResult_stg['ReportDateIssued'] = pd.to_datetime(TeraNovaTestResult_stg["ReportDateIssued"], format="%d-%b-%y")

    # Create a new dataframe to handle junk datevalue
    junk_df = pd.DataFrame()

    # This code chnages SpecimenDateTested to datatime format
    for i in TeraNovaTestResult_stg['SpecimenDateTested']:
        if len(i)!= 9:
            junk_df = junk_df.append(TeraNovaTestResult_stg.loc[TeraNovaTestResult_stg['SpecimenDateTested'] == i]).reset_index(drop=True)
            TeraNovaTestResult_stg = TeraNovaTestResult_stg.drop(labels=[TeraNovaTestResult_stg.loc[TeraNovaTestResult_stg['SpecimenDateTested'] == i].index.values[0]]
                                         , axis=0).reset_index(drop=True)

    TeraNovaTestResult_stg['SpecimenDateTested'] = pd.to_datetime(TeraNovaTestResult_stg["SpecimenDateTested"], format="%d-%b-%y")

    if junk_df.empty:
        pass
    else:
        junk_df['SpecimenDateTested'] = pd.to_datetime(junk_df["SpecimenDateTested"], format="%d-%m-%y")
        TeraNovaTestResult_stg = pd.concat([TeraNovaTestResult_stg, junk_df], ignore_index=True, sort=False)

    del junk_df   # Delete Junk dataframe to release memory


    # Convert BatchLabNumber to int
    #TeraNovaTestResult_stg['BatchLabNumber'] = TeraNovaTestResult_stg['BatchLabNumber'].astype('int')

    # Convert SpecimentAgeTest to integer
    TeraNovaTestResult_stg['SpecimenAgeTested'] = TeraNovaTestResult_stg['SpecimenAgeTested'].astype('int')


    # Remove the outbound rows
    TeraNovaTestResult_stg = TeraNovaTestResult_stg[(TeraNovaTestResult_stg['SpecimenAgeTested'] >= 1) & (TeraNovaTestResult_stg['SpecimenAgeTested'] <= age_day_max)]



    #################################### Save only the Latest Data ############################################

    TeraNovaFieldReport_stg  = TeraNovaFieldReport_stg[TeraNovaFieldReport_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == TeraNovaFieldReport_stg['ReportDateIssued']].reset_index(drop=True)

    TeraNovaTestResult_stg  = TeraNovaTestResult_stg[TeraNovaTestResult_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == TeraNovaTestResult_stg['ReportDateIssued']].reset_index(drop=True)

    # Drop NA values from the compression test report
    TeraNovaTestResult_stg = TeraNovaTestResult_stg.dropna()

    # Convert SpecimenMeasuredStrength to integer
    TeraNovaTestResult_stg['SpecimenMeasuredStrength'] = TeraNovaTestResult_stg['SpecimenMeasuredStrength'].astype('int')

    # Add a new column called CylinderTestID
    TeraNovaTestResult_stg['CylinderTestID'] = TeraNovaTestResult_stg.sort_values(['ReportDateIssued','SpecimenAgeTested'], ascending=[True,True]) \
                 .groupby(['BatchLabNumber']) \
                 .cumcount() + 1

    
    return TeraNovaFieldReport_stg,TeraNovaTestResult_stg

#### TeraNova Export Data Files Function

In [7]:
def ExportDataFiles(TeraNovaFieldReport_Final,TeraNovaTestResult_Final,TeraNovaFileLog,ExportDir):
    
    os.chdir(ExportDir)
    
    TeraNovaFieldReport_Final.to_excel('SpecimenFieldReport.xlsx', index=False)
    TeraNovaTestResult_Final.to_excel('SpecimenCompressionTestResult.xlsx', index=False)
    TeraNovaFileLog.to_excel('INGESTION_LOG.xlsx', index=False)
    engineer_report.to_excel('ProjectInformationUpdated.xlsx', index=False)

    return 'Excel Files Exported Successfully'

#### Export Job Log File Function

In [8]:
def JobLog(TeraNovaFileLog,TeraNovaFieldReport,TeraNovaTestResult,TeraNovaFieldReport_Final,TeraNovaTestResult_Final,jobTimeDiff,ExportDir):
    
    os.chdir(ExportDir)
    
    with open('JobComplete.log', 'w') as f:
        print('========================================================================================', file=f)
        print('\t\t\t\t',date.today().strftime("%A,%d %B, %Y"), file=f)
        print('\t\t\t\t       ',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
        print('========================================================================================', file=f)
        print('\t\t\t\t  TeraNova Scrapper Logs', file=f)
        print('========================================================================================', file=f)
        print('Total Files Read:', len(TeraNovaFileLog), file=f)
        print('Total Reports Read:', len(TeraNovaFieldReport), file=f)
        print('Total Test Obervations Read:', len(TeraNovaTestResult), file=f)
        print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
        print('Total Reports Ingested:', len(TeraNovaFieldReport_Final), file=f)
        print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
        print('Total Test Results Ingested:', len(TeraNovaTestResult_Final), file=f)
        print('========================================================================================', file=f)
        print('\t\t\t\t  Branson Scrapper Logs', file=f)
        print('========================================================================================', file=f)
        print('Total Files Read:', file=f)
        print('Total Reports Read:', file=f)
        print('Total Test Obervations Read:', file=f)
        print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
        print('Total Reports Ingested:', file=f)
        print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
        print('Total Test Results Ingested:', file=f)
        print('========================================================================================', file=f)
        print("\t\t\tTotal job took %d hours %d minutes %d seconds" % (jobTimeDiff.hours, jobTimeDiff.minutes, jobTimeDiff.seconds), file=f)
        print('========================================================================================', file=f)
        
    return 'Check JobComplete.log File'

## Main Code

In [9]:
# Note Down the Job Start DateTime
jobStart = datetime.now()

# Save Export Files Directory
ExportDir = os.getcwd()

# Read the Engineer's File
engineer_report = pd.read_excel('ProjectInformation.xlsx')

# generate the uuid for ProjectID
prjctid = {DCEProjectNumber: str(uuid.uuid4()) for DCEProjectNumber in engineer_report['DCEProjectNumber'].unique()}

# map uuid
engineer_report['ProjectID'] = engineer_report['DCEProjectNumber'].map(prjctid)

# Declare Datframes
TeraNovaFieldReport = pd.DataFrame()
TeraNovaTestResult = pd.DataFrame()
TeraNovaFileLog = pd.DataFrame()
TeraNovaFieldReport_Final = pd.DataFrame()
TeraNovaTestResult_Final = pd.DataFrame()

# Specify a varibale for max age days to consider in the data
age_day_max = 100

for index, row in engineer_report.iterrows():
    if (engineer_report['ScraperID'].isnull()[index]) & (engineer_report['Directory'].isnull()[index]):
        pass
    else:
        scrpr_id = engineer_report['ScraperID'][index]
        foldeR_dir = engineer_report['Directory'][index].replace('\\', '/')
        prjct_id = engineer_report['ProjectID'][index]
        
        if scrpr_id == 1001:
            FieldReport, TestResult, FileLog = TeraNova(foldeR_dir,prjct_id,scrpr_id)
            
            TeraNovaFieldReport = TeraNovaFieldReport.append(FieldReport).reset_index(drop=True) # Appends results to DF
            TeraNovaTestResult = TeraNovaTestResult.append(TestResult).reset_index(drop=True) # Appends results to DF
            TeraNovaFileLog = TeraNovaFileLog.append(FileLog).reset_index(drop=True) # Appends results to DF
            
        elif scrpr_id == 2001:
            scrpr_id = engineer_report['ScraperID'][index]
            foldeR_dir = engineer_report['Directory'][index].replace('\\', '/')
            prjct_id = engineer_report['ProjectID'][index]
            
            print("Insert Branson Code")
        else:
            pass

# Call TeraNova DataTransform Function
TeraNovaFieldReport_Final, TeraNovaTestResult_Final = TeraNovaDataTransform(TeraNovaFieldReport, TeraNovaTestResult)

# Call Export Data Files Function
ExportDataFiles(TeraNovaFieldReport_Final,TeraNovaTestResult_Final,TeraNovaFileLog,ExportDir)

# Note Down the Job Finish DateTime
jobStop = datetime.now()

# Get the JobTime Difference
jobTimeDiff = relativedelta(jobStop,jobStart)

# Call the Jog Log File Function
JobLog(TeraNovaFileLog,TeraNovaFieldReport,TeraNovaTestResult,TeraNovaFieldReport_Final,TeraNovaTestResult_Final,jobTimeDiff,ExportDir)


Insert Branson Code


'Check JobComplete.log File'

In [10]:
#(df_page_stg_new['SiteWeather'].values == '').sum()

In [11]:
#df_page_stg_new['SiteTemperature'].isna().sum()

In [12]:
#(df_page_stg_new['ConcreteSupplier'].values == '').sum()

In [13]:
#(df_page_stg_new['BatchMixID'].values == '').sum()

In [14]:
#(df_page_stg_new['Time_Molded'].values == '').sum()

In [15]:
#(df_page_stg_new['BatchSpecimenSize'].values == '').sum()

In [16]:
#df_page_stg_new['BatchTemperature'].isna().sum()

In [17]:
#df_page_stg_new['BatchUnitWeight'].isna().sum()

In [18]:
#df_page_stg_new['BatchRequiredStrength'].isna().sum()

In [19]:
#df_page_stg_new['BatchAirContent'].isna().sum()

In [20]:
#df_page_stg_new['BatchWaterAdded'].isna().sum()

In [21]:
# Add ProjectID = Unique Identifier
#engineer_report['ProjectID'] = [uuid.uuid4() for i in range(len(engineer_report.index))]